# Scraping évènements Rabat

In [1]:
import requests

In [2]:
url = 'https://www.ticket.ma/events/search?q=rabat'
response = requests.get(url)
response

<Response [200]>

In [3]:
print(response.text[:100])

<html xmlns="//www.w3.org/1999/xhtml">
	<head>
	    <meta property="fb:app_id" content="273870459404


In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

In [5]:
page_html = soup.find('div',{"class":"home clearfix local-events"})

In [6]:
ul = page_html.find('ul',class_='clearfix')

In [7]:
ul

<ul class="clearfix">
<li class="tfly-event-id- vevent" id="event-2092">
<a class="data-link" href="/event/concerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida">
<img src="/upload/events/2092/original_205_16.png"> </img></a>
<div class="activity-tools">
<p>
<a class="overlay-trigger" href="/event/concerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida">Acheter</a>
</p>
<ul>
<li><a class="activity-tools-facebook" href="javascript:void(0);" onclick="showBShareDialog('event-2092');return false">Facebook</a></li>
<li><a class="activity-tools-twitter" href="https://twitter.com/intent/tweet?text=Concerts+de+musique+classique+%22Piano%22+El+Jadida&amp;url=https%3A%2F%2Fwww.ticket.ma%2Fevent%2Fconcerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida&amp;via=ticket_ma" target="_blank">Twitter</a></li>
</ul>
</div>
<h2 class="headliners summary">
<a class="overlay-trigger" href="/event/concerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida">Concerts de musi

In [8]:
#
events = ul.find_all('a',{"class":"data-link"})

### Scraping des liens

In [9]:
liens = []

In [10]:
for a in events:
    liens.append("https://www.ticket.ma"+str(a['href']))    

In [11]:
liens

['https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida',
 'https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-07-octobre-2019-casablanca',
 'https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-07-octobre-2019-rabat',
 'https://www.ticket.ma/event/les-concerts-d-ouverture-le-11-octobre-a-theatre-national-med-v-rabat-a-20h30',
 'https://www.ticket.ma/event/jalil-tijani-jeux-de-societe-le-11-octobre-2019-au-studio-des-arts-vivants',
 'https://www.ticket.ma/event/jalil-tijani-jeux-de-societe-le-12-octobre-2019-au-studio-des-arts-vivants',
 'https://www.ticket.ma/event/sahara-music-festival-du-18-au-20-octobre-a-merzouga',
 'https://www.ticket.ma/event/le-defile-de-mode-dar-maalma-le-29-novembre-2019-au-theatre-national-mohamed-v']

### Scraping des dates (format plus interessant sur la première page)


In [12]:
d = []
for i in page_html.find_all('p',{"class":"event-date"}):
    d.append(i.get_text().replace("\t","").replace("\n",""))

In [13]:
d

['Le samedi 05 octobre 2019 · 100 Dhs',
 'Le dimanche 06 octobre 2019 · 150 Dhs',
 'Le lundi 07 octobre 2019 · 150 Dhs',
 'Le vendredi 11 octobre 2019 · 50 Dhs',
 'Le vendredi 11 octobre 2019 · 200 Dhs',
 'Le samedi 12 octobre 2019 · 200 Dhs',
 'Le vendredi 18 octobre 2019 · 1000 Dhs',
 'Le vendredi 29 novembre 2019 · 350 Dhs']

In [14]:
datez = []
for i in d:
    a,b = i.split('·')
    datez.append(a.replace("Le","").strip())

In [15]:
datez

['samedi 05 octobre 2019',
 'dimanche 06 octobre 2019',
 'lundi 07 octobre 2019',
 'vendredi 11 octobre 2019',
 'vendredi 11 octobre 2019',
 'samedi 12 octobre 2019',
 'vendredi 18 octobre 2019',
 'vendredi 29 novembre 2019']

## Formater les dates pour le format (dd.mm.yyyy)

### Fonction à utiliser

In [16]:
import datetime
import time 
import locale
locale.setlocale(locale.LC_ALL,"french")
from datetime import datetime

def format_date(num_date):
    #convertir en datetime object
    date_obj = datetime.strptime(num_date,'%A %d %B %Y')
    #formater 
    return date_obj.strftime('%d.%m.%Y')

Modifier le format des dates

In [17]:
datex = []
for i in datez:
    if "-" in i:
        x = i.split(' - ')
        print(x)
        y = []
        for j in x:
            print(format_date(j))
            y.append(format_date(j))
        datex.append(y)
    else:
        datex.append(format_date(i))

print(datex)

['05.10.2019', '06.10.2019', '07.10.2019', '11.10.2019', '11.10.2019', '12.10.2019', '18.10.2019', '29.11.2019']


In [18]:
dates = []
for i in datex:
    if len(i) >1 and type(i) ==list:
        print(" - ".join(i))
        dates.append(" - ".join(i))
    else:
        dates.append(i)

print(dates)

['05.10.2019', '06.10.2019', '07.10.2019', '11.10.2019', '11.10.2019', '12.10.2019', '18.10.2019', '29.11.2019']


## Récupérer les données lien par lien

In [19]:
from time import time
from time import sleep
from random import randint

In [20]:
start_time = time()
requests = 0

In [21]:
from requests import get

In [22]:
titres = []
descs = []
images = []
heures = []
places = []

In [23]:
for i in liens:
    print(i)
    response = get(i)
    
    timer = randint(8,15)
    #pause de loop
    print(timer)
    sleep(timer)
    
    #renvoyer un warning pour les non 'status code : 200'
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
    #parser en BS
    page_html = BeautifulSoup(response.text,'html.parser')
    
    """ Titres """
    title = page_html.find('div',class_='event-titles')
    titres.append((title.text).strip())
    
    """ Descriptions """
    detail = page_html.find('div',class_='event-details')
    descs.append((detail.find('div').text).strip())
    
    """ Images """
    img = page_html.find(id="artist-image")
    image = img.find('img')['src']
    images.append("https://www.ticket.ma"+str(image))
    
    """ Heures """
    if page_html.find('span',class_='start dtstart') is None:
        h = "n/c"
    else:
        h = (page_html.find('span',class_='start dtstart')).text
    heures.append(h[-5:])#extraire les derniers elements de la liste
    
    """ Places """
    place = page_html.find('a',class_='fancybox map-location')
    places.append(place.text)
    

print("Done")
    

https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida
14
https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-07-octobre-2019-casablanca
11
https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-07-octobre-2019-rabat
14
https://www.ticket.ma/event/les-concerts-d-ouverture-le-11-octobre-a-theatre-national-med-v-rabat-a-20h30
10
https://www.ticket.ma/event/jalil-tijani-jeux-de-societe-le-11-octobre-2019-au-studio-des-arts-vivants
9
https://www.ticket.ma/event/jalil-tijani-jeux-de-societe-le-12-octobre-2019-au-studio-des-arts-vivants
11
https://www.ticket.ma/event/sahara-music-festival-du-18-au-20-octobre-a-merzouga
10
https://www.ticket.ma/event/le-defile-de-mode-dar-maalma-le-29-novembre-2019-au-theatre-national-mohamed-v
12
Done


In [24]:
images

['https://www.ticket.ma/upload/events/2092/original_650_13.png',
 'https://www.ticket.ma/upload/events/2093/original_650_13.png',
 'https://www.ticket.ma/upload/events/2094/original_650_13.png',
 'https://www.ticket.ma/upload/events/2067/original_650_8.png',
 'https://www.ticket.ma/upload/events/2101/original_JALIL_TICKET_650x445_SDAV_DATE_1.jpg',
 'https://www.ticket.ma/upload/events/2102/original_JALIL_TICKET_650x445_SDAV_DATE_2.jpg',
 'https://www.ticket.ma/upload/events/2090/original_650.png',
 'https://www.ticket.ma/upload/events/2111/original_650_17_39_34.png']

## Tout regrouper en une liste

In [25]:
prog = []
for i,j,k,l,m,n,o in zip(titres,dates,heures,images,liens,descs,places):
    #Ajouter le lieu
    t = (i,j,k,l,m,n,o)
    prog.append(t)

In [26]:
prog

[('Concerts de musique classique "Piano" El Jadida\n\n\t\t\t\t\t\tAida Lahlou pianiste',
  '05.10.2019',
  '20h00',
  'https://www.ticket.ma/upload/events/2092/original_650_13.png',
  'https://www.ticket.ma/event/concerts-de-musique-classique-piano-le-05-octobre-2019-el-jadida',
  'L’orchestre Symphonique Royal ouvre sa 11° saison artistique avec des œuvres de deux grands maîtres de la musique classique Allemande et la participation de la jeune et talentueuse pianiste marocaine Aida LAHLOU.\nAu programme\xa0: \n-\xa0 "Concerto pour piano en la mineur, op.54 " de Robert Schumann avec Aida LAHLOU au piano\xa0\n- " Symphonie\xa0n°1 en do mineur, op.68" de Johannes Brahms.\nAida Lahlou pianiste\n"Aïda Lahlou est une pianiste pleine d\'imagination et de poésie, qui n\'hésite pas à explorer couleurs, sonorités et contrechants."(Ateş Orga, Classical Source)\nNée à Casablanca en 1998, Aïda Lahlou est l’un des talents pianistiques les plus prometteurs de sa génération. Aïda commence le piano à 

### Convertir en objet pandas

In [27]:
import pandas as pd

In [28]:
df = pd.DataFrame(prog,columns=['noms','date','heure','image','lien','description','lieu'])

In [29]:
df

,noms,date,heure,image,lien,description,lieu
0,"Concerts de musique classique ""Piano"" El Jadid...",05.10.2019,20h00,https://www.ticket.ma/upload/events/2092/origi...,https://www.ticket.ma/event/concerts-de-musiqu...,L’orchestre Symphonique Royal ouvre sa 11° sai...,Théâtre Afif
1,"Concerts de musique classique ""Piano"" Casablan...",06.10.2019,20h00,https://www.ticket.ma/upload/events/2093/origi...,https://www.ticket.ma/event/concerts-de-musiqu...,L’orchestre Symphonique Royal ouvre sa 11° sai...,Théâtre Mohammed VI
2,"Concerts de musique classique ""Piano"" Rabat\n\...",07.10.2019,20h00,https://www.ticket.ma/upload/events/2094/origi...,https://www.ticket.ma/event/concerts-de-musiqu...,L’orchestre Symphonique Royal ouvre sa 11° sai...,Théâtre National Mohammed V
3,Les Concerts d’ouverture Rabat\n\n\t\t\t\t\t\t...,11.10.2019,20h30,https://www.ticket.ma/upload/events/2067/origi...,https://www.ticket.ma/event/les-concerts-d-ouv...,"Pour la 4ème année consécutive, les concerts d...",Théâtre National Mohammed V
4,Jalil Tijani « Jeux de société »,11.10.2019,21h00,https://www.ticket.ma/upload/events/2101/origi...,https://www.ticket.ma/event/jalil-tijani-jeux-...,Après une première tournée nationale et intern...,Studio des arts vivants
5,Jalil Tijani « Jeux de société »,12.10.2019,21h00,https://www.ticket.ma/upload/events/2102/origi...,https://www.ticket.ma/event/jalil-tijani-jeux-...,Après une première tournée nationale et intern...,Studio des arts vivants
6,Sahara Music Festival,18.10.2019,n/c,https://www.ticket.ma/upload/events/2090/origi...,https://www.ticket.ma/event/sahara-music-festi...,Welcome to Sahara Music Festival by MRC Trips ...,Merzouga
7,Le Défilé Caftan Dar Maalma en cloture du cong...,29.11.2019,18h00,https://www.ticket.ma/upload/events/2111/origi...,https://www.ticket.ma/event/le-defile-de-mode-...,Le Défilé Caftan Dar Maalma est à sa troisième...,Théâtre National Mohammed V


### Serialization

In [30]:
import pickle

In [31]:
pickle_out = open("ticket.pickle","wb")
pickle.dump(prog,pickle_out)
pickle_out.close()

print("File pickled")

File pickled
